In [1]:
#import functions
import numpy as np
import scipy as sp
from scipy.integrate import odeint
import math
from numpy import linalg
import sympy

from sympy import symbols
from sympy import *
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

import plotly.graph_objects as go
import plotly.express as px

In [2]:
#defining functions
class mugas_rigid_body_functions:
  
    def __init__(self): 
        self=[];

    def simulateDySystem(self, dynamicSystemModel, Tmax, dT, X0, sysPara, figTitle, xLabel, yLabel):
        t = np.linspace(0, Tmax, int(Tmax/dT+1))
        sol = odeint(dynamicSystemModel, X0, t, args=(sysPara,));
        self.soln=sol;
        fig = go.Figure()
        [nt,nS]=sol.shape;

        # Add traces
        for nn in range(nS):
            fig.add_trace(go.Scatter(x=t, y=sol[:,nn], mode='lines+markers', name='x'+str(nn+1)))
            fig.update_layout(title=figTitle, xaxis=dict(title=xLabel),yaxis=dict(title=yLabel))
        
        fig.show()
        return [t,sol,fig] 

    def hatMatrix(self, X):
        return np.array([[0., -X[2], X[1]],[X[2], 0., -X[0]],[-X[1], X[0], 0.]])

    def qFromAxisAngles(self, theta, unitAxis):
        return np.concatenate(([np.cos(theta/2)],np.sin(theta/2)*np.array(unitAxis)))

    def RfromQuaternions(self,q):
        q0=q[0]; w=q[1:];
        R=np.identity(3)+2*q0*self.hatMatrix(w)+2*self.hatMatrix(w) @ self.hatMatrix(w);
        return R

    def rotationMatrix2EulerAngles(self,R):
        if R[2,2] <1:
            if R[2,2] > -1:
                theta2=math.acos(R[2,2]);
                theta1=math.atan2(R[0,2],R[1,2]);
                theta3=math.atan2(R[2,0],-R[2,1]);
            else: #R[2,2]=-1 No Unique solution 
                theta2=np.pi;
                theta1=-math.atan2(R[0,1],-R[0,0]);
                theta3=0;
        else: # R[2,2] = +1 No Unique solution
            theta2 = 0;
            theta1 = math.atan2(R[0,1],R[0,0]) ;
            theta3 = 0;
        return [np.pi-theta1,theta2,np.pi-theta3]

    def Re3equalsgamma(self,gamma):
        theta=math.acos(gamma[2]);
        n1=-gamma[1]/np.sin(theta);
        n2=gamma[0]/np.sin(theta);
        q00=self.qFromAxisAngles(theta,[n1,n2,0]);

        return self.RfromQuaternions(q00)

    def rotate_and_translate(self,objectVertices,R,o):
        #object vertices should be given as an numpy array of shape (3,n) 
        #Ex - X=[0, 0, 2, 2, 0, 0, 2, 2], Y=[0, 2, 2, 0, 0, 2, 2, 0], Z=[0, 0, 0, 0, 1, 1, 1, 1]; objectVertices=[X,Y,Z]
        return np.array([[o[0],o[1],o[2]]]).T + R @ objectVertices



    def addOrthNormFrame(self, fig, o, R, axisRange, axisColor):
        e1=[1,0,0]; e2=[0,1,0]; e3=[0,0,1];
        b1=R @ e1; b2=R @ e2; b3=R @ e3;
        fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b1[0])], y=[str(o[1]),str(o[1]+b1[1])], z=[str(o[2]),str(o[2]+b1[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
        fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b2[0])], y=[str(o[1]),str(o[1]+b2[1])], z=[str(o[2]),str(o[2]+b2[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
        fig.add_trace(go.Scatter3d(x=[str(o[0]),str(o[0]+b3[0])], y=[str(o[1]),str(o[1]+b3[1])], z=[str(o[2]),str(o[2]+b3[2])], hoverinfo='x+y+z', mode='lines', line=dict(width=8, color=axisColor)))
        fig.update_layout(showlegend=False, scene=dict(xaxis=dict(range=axisRange[0], autorange=False), yaxis=dict(range=axisRange[1], autorange=False), zaxis=dict(range=axisRange[2], autorange=False), aspectratio=dict(x=1, y=1, z=1)))
        return fig

    def cube_vertices(self,cubeDimensions):
        l=cubeDimensions['l']; w=cubeDimensions['w']; h=cubeDimensions['h'];
        xp=cubeDimensions['xp']; yp=cubeDimensions['yp']; zp=cubeDimensions['zp'];
        #X=[0, 0, l, l, 0, 0, l, l]; Y=[0, w, w, 0, 0, w, w, 0]; Z=[0, 0, 0, 0, h, h, h, h]; XX=[X,Y,Z];
        X=[-xp, -xp, l-xp, l-xp, -xp, -xp, l-xp, l-xp]; Y=[-yp, w-yp, w-yp, -yp, -yp, w-yp, w-yp, -yp]; Z=[-zp, -zp, -zp, -zp, h-zp, h-zp, h-zp, h-zp]; XX=[X,Y,Z];
        #X=[-l, -l, l, l, -l, -l, l, l]; Y=[-w, w, w, -w, -w, w, w, -w]; Z=[-h, -h, -h, -h, h, h, h, h]; XX=[X,Y,Z]; 
        return XX


    def animate_particle_motion(self, XX, axisRange, figTitle):
        xx=[jj[0] for jj in XX]
        yy=[jj[1] for jj in XX]
        zz=[jj[2] for jj in XX]

        fig = go.Figure(
            data=[go.Scatter3d(x=[xx[0]], y=[yy[0]],z=[zz[0]],
                            mode="markers",
                            marker=dict(color="red", size=10)),
                go.Scatter3d(x=xx, y=yy,z=zz, name='Path',
                            mode="lines",
                            line=dict(color="blue", width=2))],
            layout=go.Layout(
                scene = dict(
                            xaxis=dict(range=axisRange[0], autorange=False),
                            yaxis=dict(range=axisRange[1], autorange=False),
                            zaxis=dict(range=axisRange[2], autorange=False),
                            aspectratio=dict(x=1, y=1, z=1),
                ),
                title_text=figTitle, hovermode="closest",
                updatemenus=[dict(type="buttons",
                                buttons=[dict(label="Play",
                                                method="animate",
                                                args=[None])])]),
            frames=[go.Frame(
                data=[go.Scatter3d(
                    x=[p[0]],
                    y=[p[1]],
                    z=[p[2]],
                    name='Particle',
                    mode="markers",
                    marker=dict(color="red", size=10))])

                for p in XX]
        )

        fig.show()
        return fig

    def animated_cube_flat_shading(self, cubeVertices,figTitle):
        fig = go.Figure(
            frames=[go.Frame(data=[
            go.Mesh3d(
                # 8 vertices of a cube
                x=xx[0][0],
                y=xx[0][1],
                z=xx[0][2],
                # i, j and k give the vertices of triangles
                i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                name='y',
                opacity=0.6,
                color='#DC143C',
                flatshading = True)]) for xx in cubeVertices])

        fig.add_trace(go.Mesh3d(
                # 8 vertices of a cube
                x=cubeVertices[0][0][0],
                y=cubeVertices[0][0][1],
                z=cubeVertices[0][0][2],
                # i, j and k give the vertices of triangles
                i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                name='y',
                opacity=0.6,
                color='#DC143C',
                flatshading = True)
            )
        
        duration=10;
        fig.update_layout(
            title=figTitle,width=600,height=600,
            scene=dict(xaxis=dict(range=[-5., 5.], autorange=False),yaxis=dict(range=[-5., 5.], autorange=False),zaxis=dict(range=[-5., 5.], autorange=False),aspectratio=dict(x=1, y=1, z=1),),
            updatemenus=[dict(type="buttons",
                                buttons=[dict(label="Play",
                                                method="animate",
                                                args=[None, {"frame": {"duration": duration},"mode": "immediate","fromcurrent": True, "transition": {"duration": duration, "easing": "linear"},}]
                                                )])])
        len(fig.frames)
        fig.show()
        return fig
  
    def simulating_a_cube(self, dt, Tmax, cubeDimensions, parameters,ICs):
        XX=self.cube_vertices(cubeDimensions);
        
        #Xs=self.eulers_method(dt,Tmax,parameters,ICs);
        Xs=self.runga_kutta_method(dt,Tmax,parameters,ICs);
        ICR=ICs[0][0];
        XX0=ICR @ XX;
        
        rotatedVertices=[[XX0]]
        for X in Xs:
        #print(X[0])
           R=X[0][0]; 
           o=X[0][1];
           XXi=self.rotate_and_translate(XX,R,o);
           XX0=XXi;
           rotatedVertices+=[[XX0]];
        return rotatedVertices


    def eulers_method(self, dt,Tmax,parameters,ICs):
        M=parameters['M']; II=parameters['II'];
        invII=linalg.inv(II) ;
        timeSteps=np.arange(0,Tmax+dt,dt);
        R=ICs[0][0]; o=ICs[0][1];
        omega=ICs[1]; doto=ICs[2]; Xc=ICs[3];
        X=ICs;
        spi=R@II@R.T@omega; p=M*doto;
        Xout=[X];
        for t in timeSteps:
            [taue,fe]=externalForceModel(self, parameters,X);
            [taua,fa]=actuator(self, parameters, t, X,taue,fe);
        
            dspi=taue+taua;
            dp=fe+fa;
            #dXc=controller_dynamics(self, t,X,taue,fe,parameters)
            
            if linalg.norm(omega)>=0.0001:
                nomega=omega/linalg.norm(omega);
                thetaomegat=dt*linalg.norm(omega);
            else:
                nomega=np.array([0,0,0]);
                thetaomegat=0;
            qomegat=np.concatenate(([np.cos(thetaomegat/2)],np.sin(thetaomegat/2)*nomega));
            R=self.RfromQuaternions(qomegat) @ R;
            o=o+dt*doto;
            spi=spi+dt*dspi;
            p=p+dt*dp;
            doto=p/M;
            omega=R @ invII @ R.T @ spi;
            Xc=Xc#+dt*dXc;
            X=[[R,o],omega,doto, Xc];
            Xout+=[X]
            
        return Xout

    def runga_kutta_method(self, dt,Tmax,parameters,ICs):
        M=parameters['M']; II=parameters['II']; 
        invII=linalg.inv(II) ;
        timeSteps=np.arange(0,Tmax+dt,dt);
        X=ICs;
        Xout=[X];
        for t in timeSteps:
            Y1=self.rk4_function(0.5*dt, X, t, X, parameters);
            Y2=self.rk4_function(0.5*dt, X, t+0.5*dt, Y1, parameters);
            Y3=self.rk4_function(dt, X, t+0.5*dt, Y2, parameters);

            [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, t, X);
            [thetaomega2,nomega2,doto2,dp2,dspi2,dXc2]=self.rigid_body_system(parameters, t+0.5*dt, Y1);
            [thetaomega3,nomega3,doto3,dp3,dspi3,dXc3]=self.rigid_body_system(parameters, t+0.5*dt, Y2);
            [thetaomega4,nomega4,doto4,dp4,dspi4,dXc4]=self.rigid_body_system(parameters, t+dt, Y3);
        
            omegak=(dt/6.0)*(thetaomega1*nomega1+2*thetaomega2*nomega2+2*thetaomega3*nomega3+thetaomega4*nomega4)
            if linalg.norm(omegak)>=0.0001:
                nomegak=omegak/linalg.norm(omegak);
                thetaomegak=linalg.norm(omegak);
            else:
                nomegak=np.array([0,0,0]);
                thetaomegak=0;
            qomegak=np.concatenate(([np.cos(thetaomegak/2)],np.sin(thetaomegak/2)*nomegak));
            Rk=self.RfromQuaternions(qomegak) @ X[0][0];
            
            dotok=(1./6.0)*(doto1+2*doto2+2*doto3+doto4);
            dpk=(1./6.0)*(dp1+2*dp2+2*dp3+dp4);
            dspik=(1./6.0)*(dspi1+2*dspi2+2*dspi3+dspi4);
            dXck=(1./6.0)*(dXc1+2*dXc2+2*dXc3+dXc4);
            
            ok=X[0][1]+dt*dotok;
            pk=X[2]+dt*dpk;
            spik=X[0][0]@II@X[0][0].T@X[1]+dt*dspik;
            Xck=X[3]+dt*dXck;
            omegak=Rk @ linalg.inv(II) @ Rk.T @ spik;
            X=[[Rk,ok],omegak,pk,Xck];
            Xout+=[X]   
        return Xout

    def rk4_function(self, dtk, X, tk, Xk, parameters):
        M=parameters['M']; II=parameters['II']; 
        [thetaomega1,nomega1,doto1,dp1,dspi1,dXc1]=self.rigid_body_system(parameters, tk, Xk)
        qomega1=np.concatenate(([np.cos(dtk*thetaomega1/2)],np.sin(dtk*thetaomega1/2)*nomega1));
        R1=self.RfromQuaternions(qomega1)@X[0][0];
        p1=X[2]+dtk*dp1;
        spi1=X[0][0]@II@X[0][0].T@X[1]+dtk*dspi1; omega1=R1@linalg.inv(parameters['II'])@R1.T@spi1;
        X1=[[R1, X[0][1]+dtk*doto1],omega1,p1, X[3]+dtk*dXc1]
        return X1

    def rigid_body_system(self, parameters, t, X):
        barX=parameters['CM']; M=parameters['M'];
        R=X[0][0]; omega=X[1]; p=X[2]; 
        
        [taue,fe]=externalForceModel(self, parameters,X);
        [taua,fa]=actuator(self, parameters, t, X,taue,fe);
        
        doto=p/M; 
        dp=fe+fa;
        dspi=taue+taua;
        dXc=np.array([0.,0.,0.]); #External Dynamics #controller_dynamics(self, t,X,taue,fe,parameters)
        if linalg.norm(omega)>=0.0001:
            nomega=omega/linalg.norm(omega);
            thetaomega=linalg.norm(omega);
        else:
            nomega=np.array([0,0,0]);
            thetaomega=0;
        return [thetaomega,nomega,doto,dp,dspi,dXc]

    def animate_2D_scatter_plot(self, x, YY, xlabel, ylabel, title):
        fig = go.Figure(data=[go.Scatter(x=x,y=YY[0,:])],
                layout=go.Layout(
                    xaxis={'title':xlabel},
                    yaxis={'title':ylabel,'range':[1.1*YY.min(),1.1*YY.max()]},
                    title={'text':title,'y':0.9,'x':0.5, 'xanchor': 'center','yanchor': 'top'},
                    scene = dict(aspectratio=dict(x=1, y=1)),    
                    hovermode="closest",
                    updatemenus=[dict(type="buttons",
                                    buttons=[dict(label="Play",
                                                    method="animate",
                                                    args=[None])])]
                    ),
            frames=[go.Frame(data=[go.Scatter(x=x,y=y)]) for y in YY]
        )

        #fig.show()
        return fig 

In [3]:
#@title
def LinearSystemModel(X, t, A):
  dXdt=A @ X;
  return dXdt

In [4]:
#@title
mr=mugas_rigid_body_functions()

In [12]:
P=[1,1,1]; Q=[1,1,0]; O=[0,0,0];
PQ=np.array(P)-np.array(Q)
lengthPQ=linalg.norm(PQ)
anglePQ=np.arccos(np.array(P).dot(np.array(Q))/(linalg.norm(P)*linalg.norm(Q)))

Question 1

In [13]:
#Adding a Frame
fig = go.Figure()
fig=mr.addOrthNormFrame(fig,O,np.array([[1,0,0],[0,1,0],[0,0,1]]),[[-2,2],[-2,2],[-2,2]],'blue')
fig.show()

Question 2

In [19]:
#Adding the Point P
fig.add_trace(go.Scatter3d(x=[O[0],P[0]], y=[O[1],P[1]], z=[O[2],P[2]], hoverinfo='x+y+z', mode='lines', line=dict(width=8)))
camera=dict(
    eye=dict(x=2,y=4, z=2)
)
fig.update_layout(scene_camera=camera)
fig.show()


#Angle that OP makes with the plane spanned by the e1,12  directions
P=[1,1,1]; Q=[0,0,1]; O=[0,0,0];
PQ=np.array(P)-np.array(Q)
lengthPQ=linalg.norm(PQ)
anglePQ=np.arccos(np.array(P).dot(np.array(Q))/(linalg.norm(P)*linalg.norm(Q)))
print('Angle ='+str(90-180*anglePQ/np.pi))

Angle =35.264389682754654


Question $3$

***The assumption that time is homogeneous and that all intervals of time are inertial observer invariant means that necessarily $\tau=t+a$ where $a$ is a constant.***

The homogeneity assumption of space-time implies that 
\begin{align*}
\tau(t_1+T,x_1+a)-\tau(t_2+T,x_2+a)&=\tau(t_1,x_1)-\tau(t_2,x_2)\\
\xi(t_1+T,x_1+a)-\xi(t_2+T,x_2+a)&=\xi(t_1,x_1)-\xi(t_2,x_2)
\end{align*}
for all $a, T$ and $t_1,t_2,x_1,x_2$. 

***$\xi=\beta t+R x$ where $\beta$ is a constant $3\times 1$ matrix and $R$ is a constant $3\times 3$ matrix.***

This necessarily implies that 
\begin{align*}
\tau&=a+b t+c x,\\
\xi&=\gamma+\beta t+R x
\end{align*}
where $a,b,c,\gamma, \beta, R$ are all constant. Here $a,b\in \mathbb{R}$ and $c,\gamma, \beta\in \mathbb{R}^3$ and $R$ is a $3\times 3$ matrix.

The assumption that time is independent of space implies that $c=0$ and the assumption that all inertial observers see the same intervals of time means that necessarily  $b=1$ and  ***hence that $\tau=t+a$.***  Hence all inertial observers measure time up to an ambiguity of an additive constant and thus  without loss of generality we may assume that all observers have synchronized their clocks and hence that $a=0$. This also implies that a *universal clock* exists. 

***space intervals are inertial observer independent implies that $R$ is an orthonormal constant transformation (that is $R^TR=RR^T=I$).***

The assumption that space intervals are inertial observer independent implies that, $||\xi(t,x_1)-\xi(t,x_2)||=||x_1-x_2||$. Thus $||R (x_1-x_2)||=||x_1-x_2||$ for all $x_1, x_2$. Thus necessarily $R$ must be an orthogonal\footnote{A matrix that satisfies the properties $R^TR=RR^T=I$ is called an orthogonal transformation.} constant transformation.

***Let $O'$ be the origin of the orthonormal frame used by $\mathbf{e}'$ to make spatial measurements. If the space-time event $O'$ has the representation $(t,o)$ according to the observer $\mathbf{e}$ then since $v=\dot{o}=-R^{T}\beta=\mathrm{constant}$ we see that the velocity of the $\mathbf{e}'$ frame with respect to the $\mathbf{e}$ given by $v=\dot{o}$ must be a constant.  If both clocks of $\mathbf{e}$ and $\mathbf{e}'$ are synchronized (that is $a=0$) and if a certain space-time event $A$ has the representation $(t,x)$ according to $\mathbf{e}$ then the space-time event $A$ has the representation $(t,R(x-vt))$ according to $\mathbf{e}'$.***

Since the space is observed to be homogeneous by all inertial observers without loss of generality we may choose $\gamma=0$ (note that choosing $\gamma=0$ amounts to assuming that the origin of the spatial frames of both observers coincide at the time instant $t=0$ and does not sacrifice any generality since the space is homogeneous we can parallel translate the frames until they coincide at the time instant $t=0$).  Thus we see that the representation of the same space-time event by two different inertial observers are related by 
\begin{align*}
(\tau,\xi)=(t,\beta t+Rx).
\end{align*}

Since in the following sections we will see that the orthonormal frames are related to each by such an orthogonal transformation it follows from $\xi=Rx$ that the frame used by $\mathbf{e}'$ to make spatial measurements is also an orthonormal frame.
Let $O'$ be the origin of the orthonormal frame used by $\mathbf{e}'$. If the space-time event $O'$ has the representation $(t,o)$ according to the observer $\mathbf{e}$, it has the representation $(t,\beta t+ Ro)=(t,0)$ according to the observer $\mathbf{e}'$. Thus we have that $\beta=-R\dot{o}=-Rv$ where $v=\dot{o}$ and hence that the velocity of the origin of the $\mathbf{e}'$ frame with respect to the $\mathbf{e}$ frame, given by $v=\dot{o}=-R^T\beta$, must be a constant. **That is we see that all inertial observers must necessarily be translate at constant velocity with respect to each other without rotation.**

This also shows that the representation of a space-time event denoted by $(t,x)$ according to $\mathbf{e}$ must necessarily have the ***representation $\left(t,R(x-vt)\right)$***  for some constant $v\in \mathbb{R}^3$ according to any other inertial frame $\mathbf{e}'$. **Space appears to be homogeneous only for such observers.** In particular we can see that this is not the case for observers rotating with respect to an inertial observer $\mathbf{e}$. That is a rotating observer will not observe space to be homogeneous. 

Since $R$ is a constant, without loss of generality, one can always pick the orthonormal frame used by $\mathbf{e}'$ to be parallel to the one used by $\mathbf{e}$ so that $R=I_{3\times 3}$. Then we see that $\xi(t)=x(t)-vt$ in parallel translating inertial frames. **It is traditional to refer to parallel frames that translate at constant velocities with respect to each other as {inertial frames}.**



Question 4

Motion in $\mathbf{e}$ frame

In [ ]:
a=np.array([1,0,0]);
tt=np.linspace(0,2,11)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*a;
fig2=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e frame')


Motion in $\mathbf{e}'$ frame

In [ ]:
a=np.array([1,0,0]);
v=np.array([0,0,1])
tt=np.linspace(0,2,11)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*(a-v);
fig3=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e-prime frame')

$\vdots$ **All other inertial observers will also agree that the object is moving in a straight line at a constant velocity**

Question $5$

Motion in $\mathbf{e}'$ frame

In [ ]:
a=np.array([0,0,-1]);
tt=np.linspace(0,2,11)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*(t*a);
fig4=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e frame')

Motion in $\mathbf{e}'$ frame

In [ ]:
a=np.array([0,0,-1]);
v=np.array([1,0,0])
tt=np.linspace(0,2,11)
zeta=np.zeros((len(tt),3));
for i,t in enumerate(tt):
  zeta[i,:]=t*(t*a-v);
fig5=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e-prime frame')

$\vdots$ **Other inertial observers need not to agree that the object is moving in a straight line**

Question $6$

Motion in $\mathbf{e}'$ frame

In [ ]:
x=np.linspace(-1,1,200)
zeta=np.zeros((len(x)*2,3));
for i in range(len(x)):
  y = (1-x[i]**2)**0.5
  zeta[i][0] = x[i]
  zeta[i][1] = y
for i in range(len(x)):
  y = -(1-x[i]**2)**0.5
  zeta[len(x)+i][0] = -x[i]
  zeta[len(x)+i][1] = y
fig=mr.animate_particle_motion(zeta,[[-3,3],[-3,3],[-3,3]],'Motion in the e frame')

Motion in $\mathbf{e}'$ frame

In [ ]:
v=np.array([0,0,1])
zeta2 =np.zeros((len(x)*2,3));
tt=np.linspace(0,2,400)
for i in range(len(zeta)):
    zeta2[i,:] = zeta[i]-v*tt[i]

fig7=mr.animate_particle_motion(zeta2,[[-3,3],[-3,3],[-3,3]],'Motion in the e-prime frame')


$\vdots$ **Other inertial observers need not agree that the object is moving in a circle**